In [30]:
import psycopg2
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import unicodedata
import string
import numpy as np
import spacy    
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import warnings
warnings.filterwarnings("ignore")

In [103]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

def retrieve_all_reviews():
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews")
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df



In [104]:
df = retrieve_reviews('LG Electronics 65UJ6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')
#df = retrieve_all_reviews()

In [105]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [106]:
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']

In [107]:
df.columns = columns

In [108]:
df.head(2)

,id,p_name,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [109]:
df.groupby(['r_stars']).count()

,id,p_name,url,p_no,r_no,r_date,r_name,r_title,r_text,r_comments
r_stars,,,,,,,,,,
1.0 out of 5 stars,78,78,78,78,78,78,78,78,78,78
2.0 out of 5 stars,25,25,25,25,25,25,25,25,25,25
3.0 out of 5 stars,41,41,41,41,41,41,41,41,41,41
4.0 out of 5 stars,73,73,73,73,73,73,73,73,73,73
5.0 out of 5 stars,257,257,257,257,257,257,257,257,257,257


In [110]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','r_name','r_text']]
df_new.head(3)

,rating,r_name,r_text
0,1,Prime Customer,Now that I found the TruMotion setting so that...
1,1,GMC,Hint: Turn off Smart Picture Mode and then tur...
2,1,AncientGhost,Needed an upgrade for my 7yr old HDTV and deci...


In [111]:
nlp = spacy.load("en")
document = df_new.iloc[1]['r_text']
parsed_document = nlp(document)

In [64]:
print (parsed_document)

The scrubbing rate amongst other things are not very good on this TV. In sports the football, hockey puck or soccer ball will tick across the screen, instead of moving in a fluid motion. The quality of the picture isnt as great as I thought it would be either. Not happy with the quality of this LG, it might look like a good deal but it isnt. Very disappointed


In [112]:
for num, sentence in enumerate(parsed_document.sents):
    print (num, sentence)

0 Hint: Turn off Smart Picture Mode and then turn the Energy Saver OFF.  With Smart Picture Mode on and no access to Energy Saver setting, the picture was very dull and drab.  
1 Energy Saver reduces the backlight to save energy but at the cost of poor picture quality.  
2 After making this change, the TV picture was very bright and sharp with great detail.


In [113]:
for num, entity in enumerate(parsed_document.ents):
    print (entity, entity.label_)
token_text = [token.orth_ for token in parsed_document]
token_pos = [token.pos_ for token in parsed_document]
token_tag = [token.tag_ for token in parsed_document]

the Energy Saver OFF. ORG
Energy Saver ORG


In [114]:
token_lemma = [token.lemma_ for token in parsed_document]
token_shape = [token.shape_ for token in parsed_document]
token_prob = [token.prob for token in parsed_document]

In [116]:
df_spicy = pd.DataFrame({'token_text':token_text, 'token_lemma':token_lemma, 'token_pos':token_pos,'token_prob':token_prob, 'token_tag':token_tag})

In [117]:
df_spicy

,token_lemma,token_pos,token_prob,token_tag,token_text
0,hint,NOUN,-12.858102,NN,Hint
1,:,PUNCT,-6.128876,:,:
2,turn,VERB,-11.648442,VB,Turn
3,off,PART,-7.073742,RP,off
4,smart,PROPN,-12.219462,NNP,Smart
5,picture,PROPN,-12.162342,NNP,Picture
6,mode,PROPN,-12.467278,NNP,Mode
7,and,CONJ,-4.113108,CC,and
8,then,ADV,-6.598201,RB,then
9,turn,VERB,-8.652341,VB,turn


In [149]:
stop_words = set(stopwords.words('english'))

def punct_space(token):
    return token.is_punct or token.is_space

def is_stop_word(token):
    return token in stop_words

def is_noun(token):
    #return token.pos_ in ['NOUN','VERB','ADJ','PROPN']
    return token.pos_ in ['ADJ']


def keep(token):
    return is_noun(token) and is_stop_word(token) == False and punct_space(token) == False


def lemmatized_sentence_corpus (documents):
    for doc in documents:
        parsed_doc = nlp(doc)
        for sent in parsed_doc.sents:
            yield (' '.join([token.lemma_ for token in sent if not punct_space(token)]))


In [150]:
bigram_model = Phrases([doc.split(" ") for doc in lemmatized_sentence_corpus(df_new['r_text'].values)], min_count=1)
bigram_sentences = []
for unigram_sentence in lemmatized_sentence_corpus(df_new['r_text'].values):
    bigram_sentences.append(' '.join(bigram_model[unigram_sentence.split(" ")]))
trigram_model = Phrases([doc.split(" ") for doc in bigram_sentences], min_count=1)
type(trigram_model.vocab)

collections.defaultdict

In [120]:
' '.join(bigram_model['not','with'])

'not with'

In [169]:
# from gensim.models import Phrases
# documents = ["the mayor of york mayor was there", "machine learning can be useful sometimes","new york mayor was present"]

# sentence_stream = [doc.split(" ") for doc in documents]
# print (sentence_stream)
# bigram = Phrases(sentence_stream, min_count=1, threshold=1)
# sent = [u'the', u'of', u'new', u'york',  u'mayor', u'was', u'there']
# print(bigram[sent])

In [151]:
bigram_sentences[0]

trigram_sentences = []
for bigram_sentence in bigram_sentences:
    trigram_sentences.append(' '.join(trigram_model[bigram_sentence.split(" ")]))

trigram_sentences[6]

'hint turn_off_smart picture mode and then turn the energy_saver off. with smart picture mode_on and no access to energy_saver setting the picture be very dull and drab'

In [219]:
# trigram_reviews = LineSentence(trigram_sentences)
# trigram_dictionary = Dictionary([doc.split(" ") for doc in trigram_sentences])
# trigram_dictionary.filter_extremes(no_below=5, no_above=0.4)
# def trigram_bow_generator(trigram_sentences):
#     trigram_bow = []
#     for review in trigram_sentences:
#         print (review)
#         trigram_bow.append(trigram_dictionary.doc2bow(review))

In [26]:
#trigram_bow = trigram_bow_generator([doc.split(" ") for doc in trigram_sentences])

In [152]:
corpus = []
terms_doc = []
count = 0
for doc in df_new['r_text'].values:
    parsed_doc = nlp(doc)
    bigram_doc = ' '.join(bigram_model[(token.lemma_ for token in parsed_doc if keep(token))])
    trigram_doc = ' '.join(trigram_model[(token for token in bigram_doc.split(" "))])    
    corpus.append (trigram_doc)
    terms_doc.append(trigram_model[(token for token in bigram_doc.split(" ") if not is_stop_word(token))])

In [153]:
data = corpus

In [154]:
data[1]

'dull drab poor bright sharp great'

In [155]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 2
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(corpus)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

(335, 2)
(335, 2)
(335, 2)
[0.25820062 0.74179938]
[0.00825019 0.        ]
[ 0.03413707 -0.00270437]


In [156]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('great', 136.30690153378438), ('easy', 36.999241088435134), ('amazing', 27.641273269059152), ('happy', 20.19971126144439), ('awesome', 14.853885883442226), ('clear', 11.958079727344384), ('able', 11.543656151562718), ('excellent', 11.299179241762197), ('better', 11.132633207609876), ('good', 9.95783830643671)]
Topic 1:
[('good', 57.11278050979367), ('new', 35.60038231424002), ('smart', 25.223195350796466), ('big', 20.33341597574738), ('bad', 20.16495175274683), ('best', 14.869968341453635), ('old', 14.635124403592863), ('nice', 14.585558137967334), ('cheap', 12.06153594369391), ('right', 10.973526515146341)]
NMF Model:
Topic 0:
[('great', 3.6829557790326626), ('good', 1.255777297443153), ('easy', 0.7552712736954853), ('amazing', 0.4968013100913146), ('happy', 0.38076525688829727), ('smart', 0.379272502454899), ('big', 0.33909846115414205), ('able', 0.3313592420687511), ('little', 0.3307143116478442), ('better', 0.32753724658337724)]
Topic 1:
[('new', 3.37838026747

In [157]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [158]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(corpus))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [159]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 20
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(corpus)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

[0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.525] 1.0


In [160]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      13.712944        1       1   53.876019 -108.983932
18     10.156172        1       2  -30.316269   97.671745
10      9.096097        1       3  191.716629   -3.622257
5       7.024611        1       4 -111.087852 -161.355713
16      6.394239        1       5  137.609955   82.187447
2       5.946319        1       6   65.717407  147.357925
7       5.928554        1       7  -29.561121  186.783859
0       5.766164        1       8 -131.471725  138.393753
13      5.116521        1       9 -100.117348  -41.574139
3       4.798466        1      10   18.827475  -36.015076
12      4.005873        1      11   99.813171  -17.314842
17      3.313336        1      12 -178.711060  -87.388374
11      3.072355        1      13 -197.274658   16.014996
6       2.956688        1      14  -38.645329  -99.326744
4       2.712174        1      15  151.889206 -101.403214
15      2.603890        1      16   92.026863 -191.637314
14      2.227114        1      17   45.388210   52.242168
19      2.144825        1      18 -113.934822   49.615803
8       1.931169        1      19  -37.481705   14.483423
9       1.092490        1      20  -13.516352 -191.530106, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
25    Default  115.000000      great  115.000000  30.0000  30.0000
1     Default   31.000000    amazing   31.000000  29.0000  29.0000
31    Default   30.000000        new   30.000000  28.0000  28.0000
24    Default   55.000000       good   55.000000  27.0000  27.0000
32    Default   18.000000       nice   18.000000  26.0000  26.0000
40    Default   27.000000      smart   27.000000  25.0000  25.0000
5     Default   12.000000  beautiful   12.000000  24.0000  24.0000
17    Default   34.000000       easy   34.000000  23.0000  23.0000
18    Default   12.000000  excellent   12.000000  22.0000  22.0000
34    Default   12.000000    perfect   12.000000  21.0000  21.0000
8     Default   18.000000        big   18.000000  20.0000  20.0000
46    Default   11.000000      white   11.000000  19.0000  19.0000
33    Default   16.000000        old   16.000000  18.0000  18.0000
3     Default   14.000000    awesome   14.000000  17.0000  17.0000
26    Default   21.000000      happy   21.000000  16.0000  16.0000
6     Default   16.000000       best   16.000000  15.0000  15.0000
35    Default   10.000000    pleased   10.000000  14.0000  14.0000
12    Default   19.000000      clear   19.000000  13.0000  13.0000
37    Default   12.000000      right   12.000000  12.0000  12.0000
13    Default   13.000000       dark   13.000000  11.0000  11.0000
10    Default   10.000000     broken   10.000000  10.0000  10.0000
4     Default   20.000000        bad   20.000000   9.0000   9.0000
7     Default   13.000000     better   13.000000   8.0000   8.0000
42    Default    8.000000      sound    8.000000   7.0000   7.0000
21    Default    9.000000       fast    9.000000   6.0000   6.0000
20    Default    9.000000  fantastic    9.000000   5.0000   5.0000
41    Default    7.000000     smooth    7.000000   4.0000   4.0000
11    Default   11.000000      cheap   11.000000   3.0000   3.0000
39    Default    9.000000     simple    9.000000   2.0000   2.0000
14    Default    7.000000  defective    7.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
38    Topic20    0.164015      sharp    8.224683   0.6018  -3.9440
42    Topic20    0.166540      sound    8.552361   0.5780  -3.9287
20    Topic20    0.179845  fantastic    9.302396   0.5708  -3.8518
47    Topic20    0.173921       wide    9.005736   0.5697  -3.8853
2     Topic20    0.168761  available    8.833225   0.5589  -3.9154
36    Topic20    0.162665     remote    8.597139   0.5492  -3.9522
48    Topic20    0.160144      worth    8.527930   0.5417  -3.9678
2

In [161]:
from gensim.models import Word2Vec

In [162]:
model = Word2Vec(terms_doc, size=100, window=5, min_count=5, workers=4)

In [163]:
model.wv.most_similar(positive=['broken'])
#model.wv.index2word

[('sure', 0.20581792294979095),
 ('clear', 0.2030065506696701),
 ('bright', 0.20229607820510864),
 ('little', 0.1442437618970871),
 ('old', 0.12672077119350433),
 ('fantastic', 0.1265999972820282),
 ('worth', 0.10680564492940903),
 ('sound', 0.10584890097379684),
 ('terrible', 0.09231214970350266),
 ('disappointed', 0.0874340832233429)]

In [147]:
print ('' in stop_words)

False
